# Use Spark for R to load data and run SQL queries
This notebook introduces basic Spark concepts and helps you to start using Spark for R.

Some familiarity with R is recommended.

In this notebook, you'll use the publicly available **mtcars** data set from *Motor Trend* magazine to learn some basic R. You'll learn how to load data, create a Spark DataFrame, aggregate data, run mathematical formulas, and run SQL queries against the data.

## Table of contents
This notebook contains these main sections:

1. [Load a DataFrame](#Load_a_DataFrame)
2. [Create a Spark DataFrame](#Create_a_Spark_DataFrame)
3. [Aggregate data after grouping by columns](#Aggregate_data_after_grouping_by_columns)
4. [Operate on columns](#Operate_on_columns)
5. [Run SQL queries from the Spark DataFrame](#Run_SQL_queries_from_the_Spark_DataFrame)

<a id='Load_a_DataFrame'></a>
## 1. Load a DataFrame
A DataFrame is a distributed collection of data that is organized into named columns. The built-in R DataFrame called **mtcars** includes observations on the following 11 variables:

`[, 1]	mpg     Miles / (US) gallon`  
`[, 2]	cyl     Number of cylinders`  
`[, 3]	disp	Displacement (cu. in.)`  
`[, 4]	hp      Gross horsepower`  
`[, 5]	drat    Rear axle ratio`  
`[, 6]	wt      Weight (1000 lbs)`  
`[, 7]	qsec    1/4 mile time (seconds)`  
`[, 8]	vs      0 = V-engine, 1 = straight engine`  
`[, 9]	am      Transmission (0 = automatic, 1 = manual)`  
`[,10]	gear    Number of forward gears`  
`[,11]	carb    Number of carburetors`

Preview the first 3 rows of the DataFrame by using the head() function:

In [1]:
head(mtcars, 3)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1


Convert the car name data, which appears in the row names, into an actual column so that Spark can read it as a column:

In [2]:
mtcars$car <- rownames(mtcars)
mtcars <- mtcars[,c(12,1:11)]
rownames(mtcars) <- 1:nrow(mtcars)
head(mtcars)

car,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1


<a id='Create_a_Spark_DataFrame'></a>
## 2. Create a Spark DataFrame
Using the loaded local DataFrame, create a Spark DataFrame and print the schema, or structure, of the DataFrame:

In [3]:
library(sparklyr)
sc <- spark_connect(master = "spark://spark-master-svc:7077")

In [4]:
sdf <- sdf_copy_to(sc, mtcars, "cars", overwrite = T)

In [6]:
library(data.table)
library(dplyr)

rbindlist(sdf_schema(sdf))

name,type
car,StringType
mpg,DoubleType
cyl,DoubleType
disp,DoubleType
hp,DoubleType
drat,DoubleType
wt,DoubleType
qsec,DoubleType
vs,DoubleType
am,DoubleType


Display the content of the Spark DataFrame:

In [7]:
head(sdf)

# Source:   lazy query [?? x 12]
# Database: spark_connection
  car           mpg   cyl  disp    hp  drat    wt  qsec    vs    am  gear  carb
  <chr>       <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
1 Mazda RX4    21.0    6.  160.  110.  3.90  2.62  16.5    0.    1.    4.    4.
2 Mazda RX4 …  21.0    6.  160.  110.  3.90  2.88  17.0    0.    1.    4.    4.
3 Datsun 710   22.8    4.  108.   93.  3.85  2.32  18.6    1.    1.    4.    1.
4 Hornet 4 D…  21.4    6.  258.  110.  3.08  3.22  19.4    1.    0.    3.    1.
5 Hornet Spo…  18.7    8.  360.  175.  3.15  3.44  17.0    0.    0.    3.    2.
6 Valiant      18.1    6.  225.  105.  2.76  3.46  20.2    1.    0.    3.    1.

Try different ways of retrieving subsets of the data. For example, get the first 5 values in the **mpg** column:

In [8]:
head(select(sdf, mpg),5)

# Source:   lazy query [?? x 1]
# Database: spark_connection
    mpg
  <dbl>
1  21.0
2  21.0
3  22.8
4  21.4
5  18.7

Filter the DataFrame to retain only rows with **mpg** values that are less than 18:

In [9]:
head(filter(sdf, mpg < 18))

# Source:   lazy query [?? x 12]
# Database: spark_connection
  car           mpg   cyl  disp    hp  drat    wt  qsec    vs    am  gear  carb
  <chr>       <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
1 Duster 360   14.3    8.  360.  245.  3.21  3.57  15.8    0.    0.    3.    4.
2 Merc 280C    17.8    6.  168.  123.  3.92  3.44  18.9    1.    0.    4.    4.
3 Merc 450SE   16.4    8.  276.  180.  3.07  4.07  17.4    0.    0.    3.    3.
4 Merc 450SL   17.3    8.  276.  180.  3.07  3.73  17.6    0.    0.    3.    3.
5 Merc 450SLC  15.2    8.  276.  180.  3.07  3.78  18.0    0.    0.    3.    3.
6 Cadillac F…  10.4    8.  472.  205.  2.93  5.25  18.0    0.    0.    3.    4.

<a id='Aggregate_data_after_grouping_by_columns'></a>
## 3. Aggregate data after grouping by columns
Spark DataFrames support a number of common functions to aggregate data after grouping. For example, you can compute the average weight of cars as a function of the number of cylinders:

In [10]:
head(summarize(group_by(sdf, cyl), wtavg = avg(wt)))

# Source:   lazy query [?? x 2]
# Database: spark_connection
    cyl wtavg
  <dbl> <dbl>
1    8.  4.00
2    4.  2.29
3    6.  3.12

You can also sort the output from the aggregation to determine the most popular cylinder configuration in the DataFrame:

In [11]:
car_counts <-summarize(group_by(sdf, cyl), count = n())

In [12]:
head(arrange(car_counts, desc(count)))

# Source:     lazy query [?? x 2]
# Database:   spark_connection
# Ordered by: desc(count)
    cyl count
  <dbl> <dbl>
1    8.   14.
2    4.   11.
3    6.    7.

<a id='Operate_on_columns'></a>
## 4. Operate on columns
SparklyR provides a number of functions that you can apply directly to columns for data processing. In the following example, a basic arithmetic function converts lbs to metric tons:

In [13]:
sdf <- mutate(sdf, wtTon = wt * 60)

In [14]:
head(select(sdf, car, wt, wtTon),6)

# Source:   lazy query [?? x 3]
# Database: spark_connection
  car                  wt wtTon
  <chr>             <dbl> <dbl>
1 Mazda RX4          2.62  157.
2 Mazda RX4 Wag      2.88  172.
3 Datsun 710         2.32  139.
4 Hornet 4 Drive     3.22  193.
5 Hornet Sportabout  3.44  206.
6 Valiant            3.46  208.

<a id='Run_SQL_queries_from_the_Spark_DataFrame'></a>
## 5. Run SQL queries from the Spark DataFrame
You can register a Spark DataFrame as a temporary table and then run SQL queries over the data. The `sql` function enables an application to run SQL queries programmatically and returns the result as a DataFrame:

In [15]:
library(DBI)
highgearcars <- dbGetQuery(sc, "SELECT car, gear FROM cars WHERE gear >= 5")
head(highgearcars)

car,gear
Porsche 914-2,5
Lotus Europa,5
Ford Pantera L,5
Ferrari Dino,5
Maserati Bora,5


## That's it!
You successfully completed this notebook! You learned how to load a DataFrame, view and filter the data, aggregate the data, perform operations on the data in specific columns, and run SQL queries against the data. For more information about Spark, see the [Spark Quick Start Guide](http://spark.apache.org/docs/latest/quick-start.html).

<hr>
Copyright &copy; IBM Corp. 2017. Released as licensed Sample Materials.